In [ ]:
# Setting GPU-enabled Tensorflow
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      # tf.config.experimental.set_virtual_device_configuration(gpu, [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
      # tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
      tf.config.experimental.set_memory_growth(gpu, True)
      print("sucessful")
  except RuntimeError as e:
    print(e)

# State of mind Algorithm

In [ ]:
# pass in a dictionary of confidence score for each emotion
def get_state_of_mind(emotions_dict):
        
    confusion_list = ['neutral', 'surprise'] # duplicated combinatorial emotions for 'sad' with 'Disappointment/dissatisfaction', so 'sad' is removed from 'Confusion'
    satisfaction_delighterd_list = ['happy', 'neutral']
    disappointment_disstisfaction_list = ['neutral', 'sad']
    frustrated_list = ['sad', 'angry', 'neutral']
    state_of_mind_dict = {'Confusion': confusion_list, 
    'Satisfaction/Delighted': satisfaction_delighterd_list, 
    'Disappointment/Dissatisfaction': disappointment_disstisfaction_list,
    'Frustrated': frustrated_list}
    state_of_mind_count = {'Confusion': [0, 0], 
    'Satisfaction/Delighted': [0, 0], 
    'Disappointment/Dissatisfaction': [0, 0],
    'Frustrated': [0, 0, 0]}
    state_of_mind_list = ['Confusion', 'Satisfaction/Delighted', 'Disappointment/Dissatisfaction', 'Frustrated']
    count = 0
    state_switched = False
    # sort emotion from highest confidence score to lowest
    # sorted_emotions = emotions_dict # TODO: use for testing
    sorted_emotions = sorted(emotions_dict, key=emotions_dict.get, reverse=True)[:]
    for emotion in sorted_emotions:
        # for state in state_of_mind_dict:
        for state in state_of_mind_list:
            # if the emotion belongs to a certain state of mind
            if emotion in state_of_mind_dict[state]: 
                if count == 0:
                    state_of_mind_list.remove(state)
                    state_of_mind_list.insert(0, state)
                    state_switched = True
                # get the index of the emotion in the dictionary
                idx = state_of_mind_dict[state].index(emotion)
                # set the count to 1
                state_of_mind_count[state][idx] = 1
            
            # check if all emotions of a certain state of mind were matched
            if sum(state_of_mind_count[state]) == len(state_of_mind_count[state]):
                return state
        
        if state_switched:
            count = 1

# video face detection

import cv2
from deepface import DeepFace
import numpy as np
import os

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# read the video using VideoCapture funtion
video = cv2.VideoCapture(0)
# video = cv2.VideoCapture(os.getcwd() + "/data/clip2_7s.mp4")
# video = cv2.VideoCapture(os.getcwd() + "/data/vid2_part1.mp4")
# video = cv2.VideoCapture(os.getcwd() + "/data/vid1_short.mp4")

try:
    while video.isOpened():
    # read the video into farme
        # video.set(cv2.CAP_PROP_FPS,1) 
        ret, frame = video.read()
        # print(video.get(cv2.CAP_PROP_FPS))
        # convert the frame to grey for CascadeClassifier
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # find coordinates of faces, returns (x,y,w,h)
        # faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        faces = face_cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 5)

        for (x,y,w,h) in faces:
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            # print(frame)
            color = (255,0,0)
            try:
                analyze = DeepFace.analyze(frame[y:y+h, x:x+w], actions=['emotion'], enforce_detection=False)
                # if I set the enforce_detection as False, then though MTCNN is not detecting any face in the picture, the library will consider the whole input image as a face and compute its embedding
                # analyze = DeepFace.analyze(image, actions=['emotion'], enforce_detection=False)
                # print(analyze)
                emotion_label = analyze['dominant_emotion']
                state_of_mind_label = get_state_of_mind(analyze['emotion'])
                frame=cv2.putText(frame, emotion_label, (x, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
                frame=cv2.putText(frame, state_of_mind_label, (x, y + h + 12 ),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            except Exception as e:
                print(e)
        # frame[y1:y2, x1:x2]

        cv2.imshow("video", frame) 
        #   cv2_imshow(frame) # for google colab only
        # cv2.waitKey(5000)
        if cv2.waitKey(40) == 27:
            break

    cv2.destroyAllWindows()
    video.release

except Exception as e:
    print(e)
cv2.destroyAllWindows()
video.release